In [10]:
from ImpliedVolatility import ImpliedVolatility                                                                                 
import pandas as pd                                                                                                             
import numpy as np                                                                                                              
import datetime                                                                                                                 
                                                                                                                                
def performance(df, method, r = 0.05, sigma0 = 0.5, initialIter = 1):                                                                            
    time_start = datetime.datetime.now()                                                                                        
    dfCopy = df.copy()                                                                                                          
                                                                                                                                
    for i in df.index:                                                                                                          
        impVol = ImpliedVolatility(df.ix[i, 'StockPrice'], df.ix[i, 'StrikePrice'], df.ix[i, 'T'], r, sigma0,                   
                                   df.ix[i, 'Last'], df.ix[i, 'Type'])                                                          
                                                                                                                                
        if method == 'bisection':                                                                                               
            dfCopy.ix[i, method] = impVol.bsmBisectionVol()                                                                     
                                                                                                                                
        elif method == 'newton':                                                                                                
            dfCopy.ix[i, method] = impVol.bsmNewtonVol()                                                                        
                                                                                                                                
        elif method == 'mullerBisection':                                                                                       
            dfCopy.ix[i, method] = impVol.bsmMullerBisectionVol()                                                               
                                                                                                                                
        elif method == 'halley':                                                                                                
            dfCopy.ix[i, method] = impVol.bsmHalley()                                                                           
                                                                                                                                
        elif method == 'new_newton':                                                                                            
            impVol.bsmMullerBisectionInitial(initialIter=initialIter)                                                                                  
            dfCopy.ix[i, method] = impVol.bsmNewtonVol()                                                                        
                                                                                                                                
        elif method == 'new_halley':                                                                                            
            impVol.bsmMullerBisectionInitial(initialIter=initialIter)                                                                                  
            dfCopy.ix[i, method] = impVol.bsmHalley()     
        
        elif method == 'brentq':    
            dfCopy.ix[i, method] = impVol.bsmBrentq()
        
        elif method == 'brenth':   
            dfCopy.ix[i, method] = impVol.bsmBrenth()
        
        elif method == 'ridder':  
            dfCopy.ix[i, method] = impVol.bsmRidder()
        
        elif method == 'scipy_newton': 
            dfCopy.ix[i, method] = impVol.bsmScipyNewton()                                                                                                                                
    
    
    dfCopy.loc[:, 'log_difference'] = -np.log(np.fabs(dfCopy.IV - dfCopy.ix[:, method])) 
    dfCopy.loc[:, 'MSE'] = np.power((dfCopy.IV - dfCopy.ix[:, method]), 2) 
                                                                                                                                
    log_accuracy = dfCopy.log_difference.mean()  
    mse = dfCopy.MSE.mean()
    duration = datetime.datetime.now() - time_start
    
                                                                                                                                
    return log_accuracy, mse, duration #datetime.time(hour=0, minute=0, second=duration.seconds, microsecond=duration.microseconds).strftime("%M:%S.%f")[:-3]

In [8]:
data = pd.read_csv('europeanOptions_.01tol.csv')                                                                                       
data = data.loc[:, ['currentDate', 'ExpDate', 'StrikePrice', 'Ticker', 'Type', 'Last', 'IV', 'StockPrice', 'T']]
data.head()

,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,IV,StockPrice,T
0,2017-02-17,2017-03-17,22.5,ARR,Call,0.08,0.13956,21.709999,0.076660
1,2017-02-17,2017-03-17,20.0,ARR,Put,0.20,0.32783,21.709999,0.076660
2,2017-02-17,2017-04-21,21.0,ARR,Call,0.75,0.05176,21.709999,0.172485
3,2017-02-17,2017-04-21,22.0,ARR,Call,0.30,0.10719,21.709999,0.172485
4,2017-02-17,2017-03-17,180.0,MSG,Put,6.45,0.18380,175.300003,0.076660


In [11]:
testMethods = ['brentq', 'brenth', 'ridder', 'scipy_newton', 'bisection', 'mullerBisection', 'newton', 'halley', 'new_newton', 'new_halley']                                  
testSigma = [0.3, 0.5, 0.7, 1, 1.5, 1.7, 2]
testIter = [1, 2, 3, 5]
result = pd.DataFrame(index=testMethods, columns=['log_accuracy', 'MSE', 'Time(hh:mm:ss)'])                                                
dicResult = {}

                                                                                                                                                                                                                                                        
for iter in testIter:
    dicResult[iter] = {}
    for sigma in testSigma:  
        for method in testMethods: 
            result.loc[method, :] = performance(data, method, sigma0=sigma, initialIter=iter)
        dicResult[iter][sigma] = result.copy()                                                                                                  
        print('This is the test result for initial point sigma {:.2f}, \
initial iteration {:d} times:\n{}\n\n'.format(sigma, iter, result))

/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.0021106292083 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 4834821.20006 reached
  warnings.warn(msg, RuntimeWarning)


/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 186100.226642 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 30471.9009936 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 8972692.35162 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -1896851.51349 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.00332953061931 reached
  warnings.warn(msg, RuntimeWarning)


/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -975.944463839 reached
  warnings.warn(msg, RuntimeWarning)


/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -816096.967203 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 214.978948571 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 6777108.04707 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 23953711.0806 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.00753559953767 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 150176548.982 reached
  warnings.warn(ms

/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.00189988674422 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of 3412135.69556 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.00751754669395 reached
  warnings.warn(msg, RuntimeWarning)
/Users/Maxwell/anaconda/lib/python3.5/site-packages/scipy/optimize/zeros.py:173: RuntimeWarning: Tolerance of -0.0020715319247 reached
  warnings.warn(msg, RuntimeWarning)


...

This is the test result for initial point sigma 0.70, initial iteration 5 times:
                log_accuracy          MSE  Time(hh:mm:ss)
brentq               4.30646   0.00100102  0:00:04.725106
brenth               4.30646   0.00100102  0:00:04.700936
ridder               4.30646   0.00100102  0:00:06.485523
scipy_newton         4.12553  7.79877e+37  0:00:02.849189
bisection            4.23352      1.61306  0:00:21.526729
mullerBisection      4.23223      1.61304  0:00:22.743956
newton               4.19757  2.05935e+42  0:00:03.454922
halley                3.7735   0.00167653  0:00:01.895087
new_newton           4.17595  8.85483e+17  0:00:13.654694
new_halley            4.2429  1.79205e+07  0:00:13.532680




This is the test result for initial point sigma 1.00, initial iteration 5 times:
                log_accuracy           MSE  Time(hh:mm:ss)
brentq               4.30646    0.00100102  0:00:04.767474
brenth               4.30646    0.00100102  0:00:04.698025
ridder               4.30646    0.00100102  0:00:06.527657
scipy_newton         4.13935   1.40374e+19  0:00:02.893481
bisection            4.23352       1.61306  0:00:21.627873
mullerBisection      4.23223       1.61304  0:00:23.033174
newton               4.12668  1.55421e+181  0:00:03.291682
halley               3.64457    0.00281198  0:00:01.922577
new_newton           4.17595   8.85483e+17  0:00:13.704194
new_halley            4.2429   1.79205e+07  0:00:13.502682




This is the test result for initial point sigma 1.50, initial iteration 5 times:
                log_accuracy          MSE  Time(hh:mm:ss)
brentq               4.30646   0.00100102  0:00:04.753700
brenth               4.30646   0.00100102  0:00:04.718398
ridder               4.30646   0.00100102  0:00:06.558080
scipy_newton         4.12278   7.8153e+26  0:00:02.999224
bisection            4.23352      1.61306  0:00:21.660023
mullerBisection      4.23223      1.61304  0:00:22.889582
newton               4.07301  6.80155e+19  0:00:07.036892
halley               3.56188   0.00490749  0:00:01.923057
new_newton           4.17595  8.85483e+17  0:00:13.529563
new_halley            4.2429  1.79205e+07  0:00:13.525313




This is the test result for initial point sigma 1.70, initial iteration 5 times:
                log_accuracy           MSE  Time(hh:mm:ss)
brentq               4.30646    0.00100102  0:00:04.753743
brenth               4.30646    0.00100102  0:00:04.750648
ridder               4.30646    0.00100102  0:00:06.533123
scipy_newton         4.17953   8.81086e+31  0:00:03.021517
bisection            4.23352       1.61306  0:00:21.525027
mullerBisection      4.23223       1.61304  0:00:22.881668
newton               3.53386  6.53992e+303  0:00:02.883564
halley               3.40933    0.00586908  0:00:01.904004
new_newton           4.17595   8.85483e+17  0:00:13.755857
new_halley            4.2429   1.79205e+07  0:00:13.587763




This is the test result for initial point sigma 2.00, initial iteration 5 times:
                log_accuracy          MSE  Time(hh:mm:ss)
brentq               4.30646   0.00100102  0:00:04.736598
brenth               4.30646   0.00100102  0:00:04.739572
ridder               4.30646   0.00100102  0:00:06.534681
scipy_newton         4.11627  6.68672e+19  0:00:03.075956
bisection            4.23352      1.61306  0:00:21.475423
mullerBisection      4.23223      1.61304  0:00:22.709816
newton                3.3724  1.34356e+47  0:00:04.798405
halley                 3.136   0.00753054  0:00:01.910636
new_newton           4.17595  8.85483e+17  0:00:13.559726
new_halley            4.2429  1.79205e+07  0:00:13.493049




In [12]:
newDic = dicResult.copy()

In [6]:
for i in newDic:
    for s in newDic[i]:
        for ind in newDic[i][s].index:
            newDic[i][s].loc[ind, 'Seconds'] = float(newDic[i][s].loc[ind, 'Time(hh:mm:ss)'].split(':')[1])

In [13]:
newDic

{1: {0.3:                 log_accuracy           MSE  Time(hh:mm:ss)
  brentq               4.30646    0.00100102  0:00:04.933965
  brenth               4.30646    0.00100102  0:00:04.728438
  ridder               4.30646    0.00100102  0:00:06.583211
  scipy_newton         4.14998   5.85325e+17  0:00:02.714752
  bisection            4.23352       1.61306  0:00:21.710789
  mullerBisection      4.23223       1.61304  0:00:22.911845
  newton               4.07488  5.82465e+198  0:00:03.470157
  halley               4.22647     0.0008426  0:00:01.918691
  new_newton          -14.3382           inf  0:19:02.380316
  new_halley           -8.8698           inf  0:05:51.802205,
  0.5:                 log_accuracy           MSE  Time(hh:mm:ss)
  brentq               4.30646    0.00100102  0:00:04.731362
  brenth               4.30646    0.00100102  0:00:04.703053
  ridder               4.30646    0.00100102  0:00:06.529668
  scipy_newton         4.11276   7.04273e+23  0:00:02.793572
  bisectio

In [44]:
df_plt = pd.DataFrame(index=testMethods)
dic_plt = {}
for i in newDic:
    for s in testSigma:
        df_plt.loc[:, 'acc_{}sigma'.format(s)] = newDic[i][s].loc[:, 'Accuracy']
    for s in testSigma:
        df_plt.loc[:, 'sec_{}sigma'.format(s)] = newDic[i][s].loc[:, 'Seconds']
    dic_plt[i] = df_plt

In [152]:
df_plt2 = pd.DataFrame(index=testMethods)
for i in testIter:
    df_plt2.loc[:, 'acc_{}iter'.format(i)] = newDic[i][0.3].loc[:, 'Accuracy']
for i in testIter:
    df_plt2.loc[:, 'sec_{}iter'.format(i)] = newDic[i][0.3].loc[:, 'Seconds']       

df_plt2.iloc[:, 4:]

,sec_10iter,sec_100iter,sec_500iter,sec_1000iter
brentq,5.218,4.860,4.852,5.566
brenth,5.179,4.841,4.801,5.591
ridder,7.503,6.727,6.704,7.811
scipy_newton,2.838,2.720,2.733,3.092
bisection,22.817,21.972,22.248,25.447
mullerBisection,23.939,23.400,23.402,26.473
newton,3.595,3.519,3.546,4.101
halley,1.975,1.932,1.954,2.298
new_newton,23.002,23.643,23.868,28.836
new_halley,23.471,30.169,24.331,28.684


In [150]:
dic_plt[10].iloc[:, 8:]

,sec_0.5sigma,sec_0.7sigma,sec_1sigma,sec_1.5sigma,sec_1.7sigma,sec_2sigma
brentq,4.849,4.863,4.897,5.131,5.654,5.666
brenth,4.813,4.829,4.812,5.257,5.608,5.668
ridder,6.692,6.696,6.695,6.871,7.830,7.764
scipy_newton,2.850,2.933,2.965,3.055,3.513,3.449
bisection,22.085,22.184,22.723,22.104,24.768,25.376
mullerBisection,23.479,23.566,23.233,24.013,26.558,26.743
newton,5.763,3.582,3.355,7.492,3.389,5.264
halley,1.948,1.925,1.962,1.996,2.179,2.067
new_newton,23.801,24.088,26.810,25.723,27.294,27.546
new_halley,24.552,24.887,25.862,28.216,28.053,28.040


In [143]:
%matplotlib 
import matplotlib.pyplot as plt
# 10 times iter

df_acc = dic_plt[10].iloc[:, :7]
axes = df_acc.plot(kind='bar', subplots=True, colormap='coolwarm', title='Accuracy Comparison for Different Initial Sigma', grid=True, sort_columns=True, legend=False, figsize=(10,15))
axes[0].axvspan(5.5, 6.5, color='g', alpha=0.5)
axes[0].axvspan(-0.5, 2.5, color='grey', alpha=0.2)
for ax in axes[1:]:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(-0.5, 2.5, color='grey', alpha=0.2)

plt.savefig('acc_sigma.png')

Using matplotlib backend: MacOSX


In [153]:
df_sec = dic_plt[10].iloc[:, 8:]
axes = df_sec.plot(kind='bar', subplots=True, colormap='coolwarm',title='Running Time Comparison for Different Initial Sigma', grid=True, sort_columns=True, legend=False, figsize=(10,15))
for ax in axes:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(2.5, 3.5, color='g', alpha=0.2)

plt.savefig('sec_sigma.png')

In [128]:
df_acc = dic_plt[100].iloc[:, :7]
axes = df_acc.plot(kind='bar', subplots=True, colormap='coolwarm', title='Accuracy Comparison for 100 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
axes[0].axvspan(5.5, 6.5, color='g', alpha=0.5)
axes[0].axvspan(-0.5, 2.5, color='grey', alpha=0.2)
for ax in axes[1:]:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(-0.5, 2.5, color='grey', alpha=0.2)

plt.savefig('acc_100_iter.png')

In [130]:
df_sec = dic_plt[10].iloc[:, 8:]
axes = df_sec.plot(kind='bar', subplots=True, colormap='coolwarm',title='Running Time Comparison for 100 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
for ax in axes:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(2.5, 3.5, color='g', alpha=0.2)

plt.savefig('sec_100_iter.png')

In [132]:
df_acc = dic_plt[500].iloc[:, :7]
axes = df_acc.plot(kind='bar', subplots=True, colormap='coolwarm', title='Accuracy Comparison for 500 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
axes[0].axvspan(5.5, 6.5, color='g', alpha=0.5)
axes[0].axvspan(-0.5, 2.5, color='grey', alpha=0.2)
for ax in axes[1:]:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(-0.5, 2.5, color='grey', alpha=0.2)

plt.savefig('acc_500_iter.png')

In [135]:
df_sec = dic_plt[500].iloc[:, 8:]
axes = df_sec.plot(kind='bar', subplots=True, colormap='coolwarm',title='Running Time Comparison for 500 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
for ax in axes:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(2.5, 3.5, color='g', alpha=0.2)

plt.savefig('sec_500_iter.png')

In [136]:
df_acc = dic_plt[1000].iloc[:, :7]
axes = df_acc.plot(kind='bar', subplots=True, colormap='coolwarm', title='Accuracy Comparison for 1000 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
axes[0].axvspan(5.5, 6.5, color='g', alpha=0.5)
axes[0].axvspan(-0.5, 2.5, color='grey', alpha=0.2)
for ax in axes[1:]:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(-0.5, 2.5, color='grey', alpha=0.2)

plt.savefig('acc_1000_iter.png')

In [137]:
df_sec = dic_plt[1000].iloc[:, 8:]
axes = df_sec.plot(kind='bar', subplots=True, colormap='coolwarm',title='Running Time Comparison for 1000 Iterations', grid=True, sort_columns=True, legend=False, figsize=(10,15))
for ax in axes:
    ax.axvspan(6.5, 7.5, color='g', alpha=0.5)
    ax.axvspan(2.5, 3.5, color='g', alpha=0.2)

plt.savefig('sec_1000_iter.png')